In [1]:
import torch

In [2]:
print(torch.__version__) # to check the pytorch version

2.2.2


In [3]:
print(torch.backends.mps.is_available()) # to check if the mps is available

True


In [5]:
# creating pytorch tensors

tensor0d = torch.tensor(1)
print(tensor0d)

tensor1d = torch.tensor([1,2,3])
print(tensor1d)

tensor2d = torch.tensor([[1,2,3],[4,5,6]])
print(tensor2d)

tensor3d = torch.tensor([[1,2,3],[4,5,6],[7,8,9]])
print(tensor3d)

tensor(1)
tensor([1, 2, 3])
tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])


In [6]:
# to check the dataype of the tensor
tensor0d.dtype

torch.int64

In [ ]:
float_tensor = torch.tensor([1.0,2.0])